In [ ]:
import os
import sys
import json
import datetime

import pandas as pd
import numpy as np

In [ ]:
cwd = os.getcwd()
join = os.path.join
norm = os.path.normpath

In [ ]:
sys.path.append(norm(join(cwd, '..', '..', '..', 'glhe')))

In [ ]:
from standalone.plant_loop import PlantLoop
import glhe

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [15, 9]
plt.rcParams['font.size'] = 14

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
def rename_csv_g_files(new_name):
    for root, dirs, files in os.walk(cwd):
        for file in files:
            if file in ['g.csv', 'g_b.csv', 'sts.csv', 'lts.csv']:
                tokens = file.split('.')
                name = '{}_{}.csv'.format(tokens[0], new_name)
                if os.path.exists(join(cwd, name)):
                    os.remove(join(cwd, name))
                os.rename(file, name)

In [ ]:
d = glhe.utilities.functions.load_json('STS_Flow_input.json')
d['simulation']['output-path'] = cwd

alpha = d['soil']['conductivity'] / (d['soil']['specific-heat'] * d['soil']['density'])

ts = d['borehole-definitions'][0]['length'] ** 2 / (9 * alpha)

In [ ]:
flow_rate = 0.01

d['flow-profile'][0]['value'] = flow_rate
d['simulation']['output-csv-name'] = 'out_{:0.2f}.csv'.format(flow_rate)

with open(join(cwd, 'in_{:0.2f}.json'.format(flow_rate)), 'w') as f:
    f.write(json.dumps(d, sort_keys=True, indent=2, separators=(',', ': ')))
    
PlantLoop('in_{:0.2f}.json'.format(flow_rate)).simulate()

In [ ]:
flow_rate = 0.05

d['flow-profile'][0]['value'] = flow_rate
d['simulation']['output-csv-name'] = 'out_{:0.2f}.csv'.format(flow_rate)

with open(join(cwd, 'in_{:0.2f}.json'.format(flow_rate)), 'w') as f:
    f.write(json.dumps(d, sort_keys=True, indent=2, separators=(',', ': ')))
    
PlantLoop('in_{:0.2f}.json'.format(flow_rate)).simulate()

In [ ]:
flow_rate = 0.1

d['flow-profile'][0]['value'] = flow_rate
d['simulation']['output-csv-name'] = 'out_{:0.2f}.csv'.format(flow_rate)

with open(join(cwd, 'in_{:0.2f}.json'.format(flow_rate)), 'w') as f:
    f.write(json.dumps(d, sort_keys=True, indent=2, separators=(',', ': ')))
    
PlantLoop('in_{:0.2f}.json'.format(flow_rate)).simulate()

In [ ]:
flow_rate = 0.2

d['flow-profile'][0]['value'] = flow_rate
d['simulation']['output-csv-name'] = 'out_{:0.2f}.csv'.format(flow_rate)

with open(join(cwd, 'in_{:0.2f}.json'.format(flow_rate)), 'w') as f:
    f.write(json.dumps(d, sort_keys=True, indent=2, separators=(',', ': ')))
    
PlantLoop('in_{:0.2f}.json'.format(flow_rate)).simulate()

In [ ]:
flow_rate = 0.3

d['flow-profile'][0]['value'] = flow_rate
d['simulation']['output-csv-name'] = 'out_{:0.2f}.csv'.format(flow_rate)

with open(join(cwd, 'in_{:0.2f}.json'.format(flow_rate)), 'w') as f:
    f.write(json.dumps(d, sort_keys=True, indent=2, separators=(',', ': ')))
    
PlantLoop('in_{:0.2f}.json'.format(flow_rate)).simulate()

In [ ]:
flow_rate = 0.4

d['flow-profile'][0]['value'] = flow_rate
d['simulation']['output-csv-name'] = 'out_{:0.2f}.csv'.format(flow_rate)

with open(join(cwd, 'in_{:0.2f}.json'.format(flow_rate)), 'w') as f:
    f.write(json.dumps(d, sort_keys=True, indent=2, separators=(',', ': ')))
    
PlantLoop('in_{:0.2f}.json'.format(flow_rate)).simulate()

In [ ]:
df_001 = pd.read_csv('out_{:0.2f}.csv'.format(0.01), parse_dates=True, index_col='Date/Time')
df_005 = pd.read_csv('out_{:0.2f}.csv'.format(0.05), parse_dates=True, index_col='Date/Time')
df_010 = pd.read_csv('out_{:0.2f}.csv'.format(0.1), parse_dates=True, index_col='Date/Time')
df_020 = pd.read_csv('out_{:0.2f}.csv'.format(0.2), parse_dates=True, index_col='Date/Time')
df_030 = pd.read_csv('out_{:0.2f}.csv'.format(0.3), parse_dates=True, index_col='Date/Time')
df_040 = pd.read_csv('out_{:0.2f}.csv'.format(0.4), parse_dates=True, index_col='Date/Time')

In [ ]:
def make_g_b(df):
    df['g_b'] = (df['SingleUTubeBHGrouted:BH 1:Outlet Temp. [C]'] - df['GroundHeatExchangerSTS:GHE 1:Borehole Wall Temp. [C]']) / (10 * df['SingleUTubeBHGrouted:BH 1:BH Resist. [m-K/W]'])
    df['lntts'] = np.log(df['Elapsed Time [s]'] / ts)
    return df

In [ ]:
df_001 = make_g_b(df_001)
df_005 = make_g_b(df_005)
df_010 = make_g_b(df_010)
df_020 = make_g_b(df_020)
df_030 = make_g_b(df_030)
df_040 = make_g_b(df_040)

In [ ]:
def plot_g_b(dfs, labels):
    fig = plt.figure()
    for idx, df in enumerate(dfs):
        plt.plot(df['lntts'], df['g_b'], label=labels[idx])
    plt.legend()
    plt.savefig('gb_flow.PNG',bbox_inches='tight')
    plt.show()

In [ ]:
plot_g_b([df_001, df_005, df_010, df_020, df_030, df_040], ['0.01', '0.05', '0.10', '0.20', '0.30', '0.40'])

In [ ]:
def plot_temps(dfs, labels):
    fig = plt.figure()
    for idx, df in enumerate(dfs):
        plt.plot(df['SingleUTubeBHGrouted:BH 1:Outlet Temp. [C]'].loc['2019-01-01 00:00:00' : '2019-01-01 00:10:00'], label=labels[idx])
    plt.legend()
    plt.show()

In [ ]:
plot_temps([df_001, df_005, df_010, df_020, df_030, df_040], ['0.01', '0.05', '0.10', '0.20', '0.30', '0.40'])